### Blog
https://miroslawmamczur.pl/wykrywanie-twarzy-real-time-w-15-liniach-kodu-w-python/

### Sanity check

In [ ]:
!python --version #command line tool

In [ ]:
from platform import python_version
print(python_version())
print("* * * * *")

import sys
print("SYS.VERSION:", sys.version)
print("SYS.EXECUTABLE:", sys.executable)

### Wczytanie bibliotek i modelu
[haarcascades on Github](https://github.com/opencv/opencv/tree/master/data/haarcascades)

Whole opencv repo ~91MB

In [ ]:
import numpy as np
import cv2

print(np.__version__)
print(cv2.__version__)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

### Step 1 - przechwytywanie Video
[OpenCV Getting Started with Videos](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html#getting-started-with-videos)

In [ ]:
cap = cv2.VideoCapture(0)
while(True):
    ret, img = cap.read()
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
cap.release()
cv2.destroyAllWindows()

### Detekcja twarzy (kamera online)
Ogólnie obrazy, które widzimy, mają postać kanału RGB (czerwony, zielony, niebieski). Szary kanał jest łatwy do przetworzenia i jest mniej intensywny obliczeniowo, ponieważ zawiera tylko 1 kanał czarno-biały. Dodatkowo wczytane modele działają znacznie lepiej na odcieniach szarości.

`gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`

Mamy przechwycony obraz w odcieniu szarości. Teraz spróbujmy zlokalizować położenie twarzy na tym obrazie. Wykorzystamy wbudowaną funkcję **detectMultiScale** z załadowanego naszego obiektu **face_cascade**, która pomoże znaleźć wszystkie lokalizacje twarzy. Można to zrobić w ten sposób:

`faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))`

Wywołując ją warto podać cztery parametry:
* **image** – pierwszy (jedyny obligo) to nasz obraz
* **scaleFactor** – wytrenowany model Haar Cascade ma zdefiniowany rozmiar twarzy, która jest wykrywana na obrazie. Jednak przeskalowując obraz wejściowy, możesz zmienić rozmiar większej twarzy na mniejszą, dzięki czemu algorytm może ją wykryć. Ustalenie tego parametru na wartość scaleFactor = 1.05 oznacza zmniejszenie rozmiaru wejściowego o 5%, co zwiększa szansę na wykrycie twarzy przez model. Im większa wartość tym jakość wykrytych obiektów jest wyższa.
* **minNeighbors** – minimalna liczba sąsiadujących obszarów, które zaklasyfikowano jako posiadający porządany obiekt. Ten parametr wpływa na jakość wykrytych twarzy. Wyższa wartość powoduje mniej wykrywalności, ale o wyższej jakości.
* **minSize** – minimalny rozmiar obszaru posiadający pożądany obiekt.

Z powyższego kroku funkcja **detectMultiScale** zwraca 4 wartości – współrzędną x, współrzędną y, szerokość (w) i wysokość (h) wykrytych cechy twarzy. Na podstawie tych 4 wartości narysujemy prostokąt wokół twarzy.

`for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(190,0,0),2)`


In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(190,0,0),2)

    cv2.imshow('img',img)
    if cv2.waitKey(30) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

### Całość w 15 linijek (kamera online)

In [ ]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(190,0,0),2)

    cv2.imshow('img',img)
    if cv2.waitKey(30) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

### Dodanie efektu rozmazania (kamera online)
Teraz ogranicza Was już tylko wyobraźnia co zrobicie z zdjęciem twarzy. Wykrywanie twarzy mamy już rozpracowane. Możecie w prosty sposób napisać mechanizm do zamazywania twarzy. Wystarczy do powyższego kodu zamiast prostokąta wstawić funkcję blur do rozmazywania i to wszystko
```python
    for (x,y,w,h) in faces:        
        face = img[y:y+h, x:x+w]
        face = cv2.blur(face,((w // 5),(h // 5)))
        img[y:y+h, x:x+w] = face
```

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))

    for (x,y,w,h) in faces:        
        face = img[y:y+h, x:x+w] #cropping recognized face
        face = cv2.blur(face,((w // 5),(h // 5))) #blurring recognized face
        img[y:y+h, x:x+w] = face #adding blurred recognized face to output image

    cv2.imshow('img',img)
    if cv2.waitKey(30) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

### Wykrywanie dodatkowo oczu i uśmiechu (kamera online)
uwaga, trzeba było mocniej dobrać "sąsiadów" dla usmiechu

In [ ]:
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(190,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,190,0),2)
            
        smile = smile_cascade.detectMultiScale(roi_gray,1.5,15)
        for (sx,sy,sw,sh) in smile:
            cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,190),2)

    cv2.imshow('img',img)
    if cv2.waitKey(30) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

### Wykrywanie dodatkowo oczu i uśmiechu (z filmu video i zapisanie)

In [ ]:
cap = cv2.VideoCapture('TWOJ PLIK.mp4')

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50,50))

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(190,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,190,0),2)
            
        smile = smile_cascade.detectMultiScale(roi_gray,1.5,15)
        for (sx,sy,sw,sh) in smile:
            cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,190),2)

    cv2.imshow('img',img)
    out.write(img)
    if cv2.waitKey(5) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()